In [1]:
from IPython.core.display import HTML
from datascience import *

import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
plt.style.use('fivethirtyeight')

import networkx as nx
from networkx.algorithms import bipartite

def css_styling():
    styles = open('../notebook_styles.css', 'r').read()
    return HTML(styles)
css_styling()

In [ ]:
# These lines load the tests.
from client.api.notebook import Notebook 
hwk_sw = Notebook('hwk_small_worlds.ok')
_ = hwk_friendship.auth(inline=True)

### L&S 88-4: Social networks

# Homework 04: Small worlds

In this homework assignment, we're going to explore the concept of *small worlds*.  Small worlds have long been studied by social networks researchers, and they have also been discussed in popular culture. The rough idea is that social networks can typically be expected to have two characteristics:

* a high level of clustering
* a short average path length

A high level of clustering is consistent with the idea of triadic closure. And a short average path length is supposed to capture situations we often seem to encounter in our day to day lives: e.g., two strangers find that they have an unexpected acquaintance in common and exclaim "it's a small world!" (see the Milgram article below).

We're going to try to assess how well these two small world predictions hold up empirically. We're going to focus on the Add Health networks. We should bear in mind that the small world theory is really about very large networks, so we will be evaluating it in an unusual situation: networks of moderate size taken from children who all live in the same community.

OPTIONAL BACKGROUND READING

If you want to learn more about small-world networks, check out this article describing an early empirical study by Milgram:

* [Milgram 1967](http://measure.igpp.ucla.edu/GK12-SEE-LA/Lesson_Files_09/Tina_Wey/TW_social_networks_Milgram_1967_small_world_problem.pdf)
* [Wikipedia article: Small-world experiment](https://en.wikipedia.org/wiki/Small-world_experiment)

More recently, researchers have studied mathematical models that can produce networks with small-world properties. Here are a couple of examples:

* [Wikipedia article: Small-world network](https://en.wikipedia.org/wiki/Small-world_network)
* [Watts & Strogatz 1998](http://www.nature.com/nature/journal/v393/n6684/abs/393440a0.html)
* [Watts 1999](http://www.jstor.org/stable/10.1086/210318?seq=1#page_scan_tab_contents)

### Read in the data 

We'll use the code that we used in the labs to read the Add Health networks in.

In [ ]:
def read_add_health_network(network_id):
    """
    network_id : integer from 1 to 84
    
    read in the Add Health network corresponding to the given id number and
    return it as an undirected networkx object
    """

    # this file was downloaded from
    # http://moreno.ss.uci.edu/data.html#adhealth
    edge_file = os.path.join("..", "data", "add-health", "comm" + str(network_id) + ".dat")
    with open(edge_file, 'r') as f:
        edge_lines = f.readlines()
        
    network = nx.parse_edgelist(edge_lines, nodetype=int, data=[('activity_level', float)])
    
    # note that we call the to_undirected method to ensure we get an undirected network
    return(network.to_undirected())

number_add_health_networks = 84
add_health_networks = [read_add_health_network(x) for x in range(1,number_add_health_networks+1)]

**Question** Fill in the `average_degree` function below. It should take as its argument a `networkx` `Graph` object and it should return the average degree of that network. (This function will be useful  function below.)

In [ ]:
def average_degree(net):
    """Given a network object, return the average degree"""
    return(...)

In [ ]:
_ = hwk_sw.grade('q_helper_degree')

### Empirical distribution in the Add Health networks

First, we'll look at the empirical distribution of clustering and average path length in the Add Health networks. We'll try to get a sense for how these metrics work by looking at whether or not they seem to be related to the size (number of nodes) in a network.

**Question** Write a loop that goes through each of the 84 Add Health networks and calculates the clustering coefficient and the number of nodes in the network. (Please use the average clustering coefficient, implemented by the `average_clustering` function from the networkx package.) Store the results in a Table called `add_health_clustering` using columns called `num_nodes` and `avg_clustering_coef`.

In [ ]:
clustering = make_array()
num_nodes = make_array()

for g in ...:
    clustering = np.append(clustering, ...)
    num_nodes = np.append(num_nodes, ...)
    
add_health_clustering = Table().with_columns(['num_nodes', num_nodes,
                                              'avg_clustering_coef', clustering])
add_health_clustering

In [ ]:
_ = hwk_sw.grade('q1')

**Question** Make a histogram showing the distribution of clustering coefficients across the 84 Add Health networks.

In [ ]:
...

**Question** Make a scatter plot that compares the number of nodes in each network (x axis) to the clustering coefficient (y axis). Does it look like the clustering coefficient changes as the number of nodes does?

In [ ]:
...

<div class='response'>
[Answer here]
</div>

### Average path length of biggest component

Remember that it really only makes sense to think about the average path length between two nodes that are in the same component. (Nodes in different components have no path between them.) Since some of the Add Health networks have more than one component, we'll start by picking out only the largest component in each network.

In [ ]:
def get_biggest_component(network):
    biggest = max(nx.connected_component_subgraphs(network), key=len)
    return(biggest)

add_health_biggest_components = [get_biggest_component(g) for g in add_health_networks]

**Question** Write a loop that goes through the largest component of each of the 84 Add Health networks and calculates the average shortest path length and the number of nodes in the network. Store the results in a Table called `add_health_sp` using columns called `num_nodes` and `avg_shortest_path`.

In [ ]:
## NOTE: your code might take a little while
##       (~ 3-5 minutes) to run

avg_shortest_path = make_array()
num_nodes = make_array()

avg_degree = make_array()

for c in ...:
    avg_shortest_path = np.append(avg_shortest_path, ...)
    num_nodes = np.append(num_nodes, ...)
    avg_degree = np.append(avg_degree, ...)

add_health_sp = Table().with_columns(['num_nodes', num_nodes,
                                      'avg_shortest_path', avg_shortest_path,
                                      'avg_degree', avg_degree])

add_health_sp

In [ ]:
_ = hwk_sw.grade('q2_b')

**Question** Plot a histogram showing the distribution of average shortest path lengths across the 84 Add Health networks' largest components.

In [ ]:
...

**Question** Make a scatter plot that compares the number of nodes in each largest component (x axis) to the average shortest path (y axis). Does it look like the average shortest path changes as the number of nodes does?

In [ ]:
...

<div class='response'>
[Answer here]
</div>

### Do real networks agree with the small-world hypothesis?

The small world theory says that a social network should have a large clustering coefficient and a small average path length. But what do large and small mean? In other words, what should we think about comparing these networks to?

Now we'll use Erdos-Renyi random networks as a *null model*. For our purposes, ER models have a couple of important properties. They have:

* short average path lengths
* very small clustering

So putting this together with the small world properties, we would expect that a small world network would have

* average path lengths that are about the same as an ER random network
* clustering that is much larger than an ER random network

We could try to study this question mathematically (as some of the networks researchers have). That is, we could come up with a mathematical model for a small world network and compare that model to the ER model. We're going to take a different approach in this homework: instead of using math, we're going to study the properties of the ER model using simulation. Specifically, we're going to

* pick one specific Add Health network to test
* generate ER networks that 'match' that specific Add Health network
* compare the clustering coefficient / average path lengths of the ER networks to the ones we observe in the Add Health network; if these agree, then we conclude that the model provides a good description of the observed data. If not, then we conclude that the data are not well-described by the model.


Let's pick out one particular Add Health network to focus on for this part.

In [ ]:
# the specific Add Health network we'll look at
ahn = add_health_networks[17]

... and let's also use a couple of functions that we created in Lab 06.

In [ ]:
def er_by_degree(n, avg_degree):
    
    return(nx.erdos_renyi_graph(n=n, p=avg_degree / (n-1)))

def rand_er_network(network):
    """
    Return a random network generated from the configuration model using
    the degree sequence of the network passed in
    """
    network_n = network.number_of_nodes()
    network_dbar = average_degree(network)
    return(er_by_degree(network_n, network_dbar))

### Developing a simulation from a null model

**Question** Describe what the function `er_by_degree` does.  
*[HINT: You may find Lab 06 helpful to review.]*

<div class='response'>
[Answer here]
</div>

**Question** Write a function which, given a network, returns its average shortest path length. If the network has more than one component, your function should return the average path length in the biggest component.

In [ ]:
def avg_path_length(net):
    if nx.number_connected_components(net) > 1:
        net = ...
    
    return(...)

In [ ]:
_ = hwk_sw.grade('q3_0')

**Question** Write a simulation that generates 500 Erdos Renyi random networks that match the Add Health network `ahn`. (By 'match', we mean that the ER network should have the same average degree and number of nodes as the Add Health network `ahn`.). For each generated ER network, calculate the average clustering and use the function you wrote above to calculate the average path length. Store the results in a table called `er_res`.  
*NOTE: this may take a few minutes (say 3) to run*

In [ ]:
observed_apl = avg_path_length(ahn)
observed_cc = nx.average_clustering(ahn)

er_cc = make_array()
er_apl = make_array()

# the underscore (_) means that we don't
# care which iteration is which -- we just want
# to repeat this 500 times
for _ in range(500):
    
    er_net = ...
    er_cc = np.append(er_cc, ...)
    er_apl = np.append(er_apl, ...)
    
er_res = Table().with_columns('cc', er_cc,
                              'apl', er_apl)

er_res

In [ ]:
_ = hwk_sw.grade('q3_a')

**Question** Now print out the observed average path length in the Add Health network `ahn` and plot a histogram of the average path lengths in the ER networks you just simulated.

In [ ]:
...
...

**Question** Where would the observed Add Health network's average path length fall in the Erdos Renyi networks' distribution?

<div class='response'>
[Answer here]
</div>

**Question** Now print out the observed clustering coefficient in the Add Health network `ahn` and plot a histogram of the clustering coefficients in the ER networks you just simulated.

In [ ]:
...
...

**Question** Where would the observed Add Health network's clustering coefficient fall in the Erdos Renyi networks' distribution?

<div class='response'>
[Answer here]
</div>

**Question** You just looked at the relationship between (1) the predicted average path length in an Erdos-Renyi random network and the observed relationship in the Add Health network; and (2) the predicted average clustering coefficient in an Erdos-Renyi random network and the observed relationship in the Add Health network. Putting these two findings together, what do your results suggest about the small world hypothesis? Do you think the small world hypothesis seem to hold in this Add Health network?

<div class='response'>
[Answer here]
</div>

## Tests

In [ ]:
# this cell runs all the tests at once!
print("Running all tests...")
_ = [hwk_sw.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]
print("Finished running all tests.")

# Please hand this homework in using two different methods

Both submissions must be completed by **midnight on Tuesday, October 24th.**<BR>
**Late homework will not be accepted**, so please be sure to hand in as much as you have finished by the deadline. Good luck!

**FIRST, please run the following cell to submit using `okpy`**

In [ ]:
_ = hwk_sw.submit()

**SECOND** Please hand this homework in as a `.pdf` file on Bcourses. 